In [1]:
import json
file_path="/home/v-leiwang8/LLaMA-Factory/data/reward_data/examples.json"
with open(file_path,'r') as f:
    data=json.load(f)
print(len(data))

77


In [ ]:
from fastapi import FastAPI, Request
from transformers import AutoTokenizer, AutoModelForCausalLM,AutoModel
from trl import AutoModelForCausalLMWithValueHead
import uvicorn, json, datetime
import torch
import os
from tqdm import tqdm
import numpy as np

In [2]:
model_dir = "THUDM/chatglm3-6b"
adapter_path="/home/v-leiwang8/LLaMA-Factory/saves/ChatGLM/lora/point/checkpoint-1000"
from models.glm3_reward.modeling_chatglm import ChatGLMRM
from models.glm3_reward.tokenization_chatglm import ChatGLMTokenizer
# model_dir="/home/v-leiwang8/ChatGLM3/finetune_demo/output/lora_voldemort/checkpoint-8000"
tokenizer = ChatGLMTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = ChatGLMRM.from_pretrained(model_dir, trust_remote_code=True).half().cuda()
print(model)


Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.
Loading checkpoint shards: 100%|██████████| 7/7 [00:01<00:00,  3.95it/s]


ChatGLMRM(
  (pretrained_model): ChatGLMForConditionalGeneration(
    (transformer): ChatGLMModel(
      (embedding): Embedding(
        (word_embeddings): Embedding(65024, 4096)
      )
      (rotary_pos_emb): RotaryEmbedding()
      (encoder): GLMTransformer(
        (layers): ModuleList(
          (0-27): 28 x GLMBlock(
            (input_layernorm): RMSNorm()
            (self_attention): SelfAttention(
              (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)
              (core_attention): CoreAttention(
                (attention_dropout): Dropout(p=0.0, inplace=False)
              )
              (dense): Linear(in_features=4096, out_features=4096, bias=False)
            )
            (post_attention_layernorm): RMSNorm()
            (mlp): MLP(
              (dense_h_to_4h): Linear(in_features=4096, out_features=27392, bias=False)
              (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
            )
          )
  

In [11]:
def get_reward(prompt):
    input_ids = tokenizer.encode(text=prompt, add_special_tokens=False) + [tokenizer.eos_token_id]
    if len(input_ids) > 8192:
        input_ids = input_ids[-8192:]
    input_ids = torch.tensor(input_ids).unsqueeze(0).cuda()
    with torch.no_grad():
        score = model(input_ids=input_ids)[2].item() * 4 + 1
    return score

In [8]:
file_path="/home/v-leiwang8/LLaMA-Factory/data/reward_data/evaluation.json"
with open(file_path,'r',encoding='utf-8') as f:
    data=json.load(f)

ans=[]
labels=[]
num=1000
for i in tqdm(range(num)):
    labels.append(data[i]['output'])
    prompt=data[i]['instruction']
    input_ids = tokenizer.encode(text=prompt, add_special_tokens=False) + [tokenizer.eos_token_id]
    if len(input_ids) > 8192:
        input_ids = input_ids[-8192:]
    input_ids = torch.tensor(input_ids).unsqueeze(0).cuda()
    with torch.no_grad():
        score = model(input_ids=input_ids)[2].item() * 4 + 1
        ans.append(score)
        #print(score)
print(ans)
ans=np.array(ans)
labels=np.array(labels)
print(ans.mean(),ans.max(),ans.min())
print(labels.mean(),labels.max(),labels.min())
rmse=np.sqrt(((ans-labels)**2).mean())
print(rmse)

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [01:22<00:00, 12.09it/s]

[1.2249755859375, 1.331298828125, 3.5703125, 2.240234375, 1.3603515625, 1.267333984375, 2.8779296875, 1.228271484375, 1.537109375, 1.05029296875, 1.13232421875, 1.49169921875, 1.1702880859375, 1.6259765625, 1.144287109375, 1.6337890625, 1.98779296875, 1.83984375, 2.30078125, 2.048828125, 1.1990966796875, 3.27734375, 1.286865234375, 1.06561279296875, 1.304443359375, 1.2164306640625, 2.2529296875, 1.1346435546875, 1.34228515625, 1.283203125, 2.5400390625, 1.32666015625, 1.1883544921875, 2.5009765625, 1.56787109375, 1.599609375, 1.2303466796875, 1.228759765625, 1.12432861328125, 1.7666015625, 1.113525390625, 1.446533203125, 1.479736328125, 1.623046875, 1.06951904296875, 1.98193359375, 1.2005615234375, 1.59765625, 1.293212890625, 1.33251953125, 1.31689453125, 1.259033203125, 2.2109375, 1.12841796875, 1.1715087890625, 1.308349609375, 1.1484375, 1.3134765625, 1.25341796875, 1.158203125, 1.34033203125, 1.22998046875, 1.204345703125, 1.56396484375, 1.2139892578125, 1.7958984375, 1.1748046875, 

In [14]:
import csv
import pandas as pd
file_path="/home/v-leiwang8/RoleAgent/output/data/reward/evaluation_detail.json"
with open(file_path,'r',encoding='utf-8') as f:
    data=json.load(f)

csv_path="reward_result.csv"
fieldnames = ['Title','Judger','Narrator','Model', 'Knowledge Accuracy', 'Emotional Expression', 'Personality Traits', 'Behavioral Accuracy', 'Immersion', 'Adaptability', 'Behavioral Coherence','Average']
if not os.path.exists(csv_path):
    with open(csv_path, mode='w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()

#record=pd.DataFrame(columns=fieldnames)

zh_titles=['西游记','三国演义','红楼梦', '还珠格格', '笑傲江湖']
en_titles=['Harry_Potter','The_Lord_of_the_Rings',  'The_Matrix', 'Twilight','A_Song_of_Ice_and_Fire' ]
records=[]
#zh_record
#en_record={}
for i in range(len(data)):
    prompt=data[i]['instruction']
    input_ids = tokenizer.encode(text=prompt, add_special_tokens=False) + [tokenizer.eos_token_id]
    if len(input_ids) > 8192:
        input_ids = input_ids[-8192:]
    input_ids = torch.tensor(input_ids).unsqueeze(0).cuda()
    with torch.no_grad():
        score = model(input_ids=input_ids)[2].item() * 4 + 1
    criterion=data[i]['criteria']
    temp={'Title':data[i]['title'],'Judger':data[i]['judger'],'Narrator':data[i]['narrator'],'Model':data[i]['model'],'Knowledge Accuracy':0,'Emotional Expression':0,'Personality Traits':0,'Behavioral Accuracy':0,'Immersion':0,'Adaptability':0,'Behavioral Coherence':0,'Average':0}
    temp[criterion]=score
    records.append(temp)


    

        #print(score)
    

[{'Title': '西游记', 'Judger': 'azure_GPT-4-Turbo', 'Narrator': 'azure_gpt-35-turbo', 'Model': 'azure_GPT-4-Turbo', 'Knowledge Accuracy': 2.177734375, 'Emotional Expression': 0, 'Personality Traits': 0, 'Behavioral Accuracy': 0, 'Immersion': 0, 'Adaptability': 0, 'Behavioral Coherence': 0, 'Average': 0}, {'Title': '西游记', 'Judger': 'azure_GPT-4-Turbo', 'Narrator': 'azure_gpt-35-turbo', 'Model': 'azure_GPT-4-Turbo', 'Knowledge Accuracy': 0, 'Emotional Expression': 2.74609375, 'Personality Traits': 0, 'Behavioral Accuracy': 0, 'Immersion': 0, 'Adaptability': 0, 'Behavioral Coherence': 0, 'Average': 0}, {'Title': '西游记', 'Judger': 'azure_GPT-4-Turbo', 'Narrator': 'azure_gpt-35-turbo', 'Model': 'azure_GPT-4-Turbo', 'Knowledge Accuracy': 0, 'Emotional Expression': 0, 'Personality Traits': 2.0556640625, 'Behavioral Accuracy': 0, 'Immersion': 0, 'Adaptability': 0, 'Behavioral Coherence': 0, 'Average': 0}, {'Title': '西游记', 'Judger': 'azure_GPT-4-Turbo', 'Narrator': 'azure_gpt-35-turbo', 'Model': 'a

In [22]:

df = pd.DataFrame(records)

# 为每条记录计算评估指标的平均值
metrics = ['Knowledge Accuracy', 'Emotional Expression', 'Personality Traits', 'Behavioral Accuracy', 'Immersion', 'Adaptability', 'Behavioral Coherence']
df[metrics] = df[metrics] * 7
df['Average'] = df[metrics].mean(axis=1)

# 创建两个场景的DataFrame
zh_df = df[df['Title'].isin(zh_titles)]
en_df = df[df['Title'].isin(en_titles)]

# 计算两种场景下各模型的指标平均值
zh_model_means = zh_df.groupby(['Judger','Narrator','Model'])[metrics].mean()
en_model_means = en_df.groupby(['Judger','Narrator','Model'])[metrics].mean()
zh_model_means['Average'] = zh_model_means[metrics].mean(axis=1)
en_model_means['Average'] = en_model_means[metrics].mean(axis=1)
# 输出结果查看
print("中文场景各模型指标平均值:")
print(zh_model_means)
print("\n英文场景各模型指标平均值:")
print(en_model_means)

# 如果需要，也可以将这些结果保存到CSV文件
zh_model_means.to_csv('zh_model_averages.csv')
en_model_means.to_csv('en_model_averages.csv')

中文场景各模型指标平均值:
                                                               Knowledge Accuracy  \
Judger            Narrator           Model                                          
azure_GPT-4-Turbo azure_gpt-35-turbo Baichuan2-13B-Chat                  1.498053   
                                     Baichuan2-7B-Chat                   1.647404   
                                     Meta-Llama-3-8B-Instruct            4.073027   
                                     Mistral-7B-Instruct-v0.2            1.752246   
                                     Phi-3-mini-128k-instruct            1.663832   
                                     Qwen1.5-14B-Chat                    1.619859   
                                     Qwen1.5-7B-Chat                     1.794374   
                                     azure_GPT-4-Turbo                   1.375070   
                                     azure_gpt-35-turbo                  1.434497   

                                                  

: 